In [63]:
import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings('ignore')

## New:

In [64]:
data2 = pd.read_csv("../data/data2.csv", parse_dates=['date'])

In [65]:
# retrieve unique user data named as user_date

user_data = pd.DataFrame(data2["user_id"].unique(), columns = ["user_id"])
# sample the user data with frac = 1%
user_data = user_data.sample(frac=0.05 ,axis=0)

user_data

,user_id
1383934,606823634
1197725,597122624
1244424,599419600
1583321,618564553
1195667,597014632
...,...
1041561,587236531
265343,521711863
1309054,602491901
1540575,615651775


In [66]:
# retrieve the uer_id and date for each session
data_new = pd.DataFrame(data2["user_id"])
data_new["date"] = data2["date"]
# create L1
data_new["L1"] = 1

data_new

,user_id,date,L1
0,465496,2020-01-14,1
1,1120748,2019-11-04,1
2,1180452,2019-12-28,1
3,1458813,2019-11-19,1
4,1458813,2019-11-20,1
...,...,...,...
2804615,622090016,2020-02-29,1
2804616,622090043,2020-02-29,1
2804617,622090052,2020-02-29,1
2804618,622090098,2020-02-29,1


In [67]:
# join the all user_date table with sampled user table
data_new = pd.merge(data_new, user_data, how = "inner", on = "user_id")

In [68]:
data_new

,user_id,date,L1
0,1180452,2019-12-28,1
1,4661182,2019-12-06,1
2,4661182,2019-12-22,1
3,4661182,2020-01-28,1
4,4661182,2020-02-15,1
...,...,...,...
141885,622080637,2020-02-29,1
141886,622081281,2020-02-29,1
141887,622082232,2020-02-29,1
141888,622087993,2020-02-29,1


In [69]:
# create a table as time series data with time from 10-01-2019 to 02-29-2020
time_range = pd.DataFrame(pd.date_range(start='10/01/2019', end='02/29/2020'), columns = ["date"])

In [70]:
# cross product the time and user data
# so each user be a time series data
from pandas import merge
user_data['key'] = 0
time_range['key'] = 0

time_user_data = merge(user_data, time_range, on='key')[['user_id', 'date']]


time_user_data

,user_id,date
0,606823634,2019-10-01
1,606823634,2019-10-02
2,606823634,2019-10-03
3,606823634,2019-10-04
4,606823634,2019-10-05
...,...,...
12457611,605770601,2020-02-25
12457612,605770601,2020-02-26
12457613,605770601,2020-02-27
12457614,605770601,2020-02-28


In [71]:
# join table
time_user_data = pd.merge(time_user_data, data_new, how = "outer", on = ["user_id", "date"])

In [72]:
# fill NULL value with 0
time_user_data["L1"] = time_user_data["L1"].fillna(value = 0)

# change datatype from float to int
time_user_data["L1"] = time_user_data["L1"].astype(int)

In [73]:
time_user_data

,user_id,date,L1
0,606823634,2019-10-01,0
1,606823634,2019-10-02,0
2,606823634,2019-10-03,0
3,606823634,2019-10-04,0
4,606823634,2019-10-05,0
...,...,...,...
12457611,605770601,2020-02-25,0
12457612,605770601,2020-02-26,0
12457613,605770601,2020-02-27,0
12457614,605770601,2020-02-28,0


In [74]:
# create L7
time_user_data["L7"] = time_user_data.groupby(["user_id"])["L1"].transform(lambda x:x.rolling(7, min_periods = 1).sum())

In [75]:
time_user_data

,user_id,date,L1,L7
0,606823634,2019-10-01,0,0
1,606823634,2019-10-02,0,0
2,606823634,2019-10-03,0,0
3,606823634,2019-10-04,0,0
4,606823634,2019-10-05,0,0
...,...,...,...,...
12457611,605770601,2020-02-25,0,0
12457612,605770601,2020-02-26,0,0
12457613,605770601,2020-02-27,0,0
12457614,605770601,2020-02-28,0,0


In [76]:
# get first date per user
min_dates = time_user_data[time_user_data['L1'] > 0].\
            groupby('user_id')['date'].min().\
            reset_index().\
            rename(columns={"date": "first_date"})
min_dates['first_date'] = pd.to_datetime(min_dates['first_date'])
min_dates

,user_id,first_date
0,1180452,2019-12-28
1,4661182,2019-12-06
2,10120387,2020-02-04
3,11253733,2020-01-30
4,27085410,2019-10-06
...,...,...
81953,622080637,2020-02-29
81954,622081281,2020-02-29
81955,622082232,2020-02-29
81956,622087993,2020-02-29


In [77]:
# join first date by user back to initial DF
time_user_data = pd.merge(time_user_data, min_dates, how='inner', on='user_id')
time_user_data = time_user_data[time_user_data['date'] > time_user_data['first_date']]
time_user_data

,user_id,date,L1,L7,first_date
119,606823634,2020-01-28,0,1,2020-01-27
120,606823634,2020-01-29,0,1,2020-01-27
121,606823634,2020-01-30,0,1,2020-01-27
122,606823634,2020-01-31,0,1,2020-01-27
123,606823634,2020-02-01,0,1,2020-01-27
...,...,...,...,...,...
12457611,605770601,2020-02-25,0,0,2020-01-25
12457612,605770601,2020-02-26,0,0,2020-01-25
12457613,605770601,2020-02-27,0,0,2020-01-25
12457614,605770601,2020-02-28,0,0,2020-01-25


In [78]:
# for those who churned, establish churn date as 1st day of L7==0 (after start date)
churners = pd.DataFrame(time_user_data[time_user_data['L7'] == 0].groupby('user_id')['date'].min()).\
           reset_index().\
           rename(columns={"date": "churn_date"})
churners['churn_date'] = pd.to_datetime(churners['churn_date'])
churners

,user_id,churn_date
0,1180452,2020-01-04
1,4661182,2019-12-13
2,10120387,2020-02-11
3,11253733,2020-02-06
4,27085410,2019-10-13
...,...,...
78219,619010313,2020-02-29
78220,619013581,2020-02-29
78221,619014157,2020-02-29
78222,619019547,2020-02-29


In [80]:
# join churn info for churners back to initial DF
user_c = pd.merge(time_user_data, churners, how='inner', on='user_id')
user_c['days_survived'] = (user_c['churn_date'] - user_c['first_date']).dt.days
user_c['churn_status'] = 1
user_c = user_c[['user_id','first_date','churn_date','days_survived','churn_status']].drop_duplicates()

user_c

,user_id,first_date,churn_date,days_survived,churn_status
0,606823634,2020-01-27,2020-02-03,7,1
33,597122624,2020-01-05,2020-01-12,7,1
88,599419600,2020-01-10,2020-01-17,7,1
138,618564553,2020-02-21,2020-02-29,8,1
146,597014632,2020-01-05,2020-01-12,7,1
...,...,...,...,...,...
6465286,587236531,2019-12-15,2019-12-22,7,1
6465362,521711863,2019-10-19,2019-10-26,7,1
6465495,602491901,2020-01-17,2020-01-24,7,1
6465538,615651775,2020-02-16,2020-02-23,7,1


In [81]:
not_churners = time_user_data.groupby('user_id').filter(lambda s: not any(s.L7 == 0)).\
               groupby('user_id')['date'].max().reset_index().\
               rename(columns={"date": "churn_date"})
not_churners

,user_id,churn_date
0,95431155,2020-02-29
1,110559851,2020-02-29
2,121766217,2020-02-29
3,165183560,2020-02-29
4,209657701,2020-02-29
...,...,...
3224,621667836,2020-02-29
3225,621668949,2020-02-29
3226,621669139,2020-02-29
3227,621671280,2020-02-29


In [82]:

# join non-churner info back to initial DF
user_nc = pd.merge(time_user_data, not_churners, how='inner', on='user_id')
user_nc['days_survived'] = (user_nc['churn_date'] - user_nc['first_date']).dt.days + 1
user_nc['churn_status'] = 0
user_nc = user_nc[['user_id','first_date','churn_date','days_survived','churn_status']].drop_duplicates()

user_nc

,user_id,first_date,churn_date,days_survived,churn_status
0,620362126,2020-02-25,2020-02-29,5,0
4,620508498,2020-02-26,2020-02-29,4,0
7,588118702,2020-02-22,2020-02-29,8,0
14,472504867,2020-02-28,2020-02-29,2,0
15,620949498,2020-02-27,2020-02-29,3,0
...,...,...,...,...,...
13846,621148035,2020-02-27,2020-02-29,3,0
13848,619195333,2020-02-23,2020-02-29,7,0
13854,539567348,2020-02-25,2020-02-29,5,0
13858,616362232,2020-02-22,2020-02-29,8,0


In [83]:
#union final non-churner info and churner info into a single "life table" DF
user_data_final = pd.concat([user_c, user_nc])
user_data_final

,user_id,first_date,churn_date,days_survived,churn_status
0,606823634,2020-01-27,2020-02-03,7,1
33,597122624,2020-01-05,2020-01-12,7,1
88,599419600,2020-01-10,2020-01-17,7,1
138,618564553,2020-02-21,2020-02-29,8,1
146,597014632,2020-01-05,2020-01-12,7,1
...,...,...,...,...,...
13846,621148035,2020-02-27,2020-02-29,3,0
13848,619195333,2020-02-23,2020-02-29,7,0
13854,539567348,2020-02-25,2020-02-29,5,0
13858,616362232,2020-02-22,2020-02-29,8,0


In [ ]:
user_data_final.to_csv("../data/user_data.csv")